### Groups
A group is an algebraic structure consisting of a set $G$ equipped with a binary operation $\cdot$ that combines any two elements $a, b \in G$ to form another element $a \cdot b$ (Noted as $(G, \cdot)$). To be considered a group, the structure must satisfy four axioms:
1. Closure: For all $a, b \in G$, $a * b \in G$.
2. Associativity: For all $a, b, c \in G$, $(a * b) * c = a * (b * c)$.
3. Identity Element: There exists an element $e \in G$ such that $a * e = e * a = a$.
4. Inverse Element: For every $a \in G$, there exists an element $b \in G$ such that $a * b = b * a = e$

Two of the most critical starting points in group theory are the uniqueness theorems. We state that the identity element $e$ in a group is unique; there cannot be two distinct elements that both act as the identity. Furthermore, for every element $a$ in the group, there exists a unique inverse $a^{-1}$ such that $a \cdot a^{-1} = e$. These properties ensure that group operations are well-defined and reversible.

Lean Reference: See theorem identity_is_unique and theorem inverse_is_unique in AA_Proofs.lean.

        Include HW (6c) exercise: 
        Show that if $a,b \in G$ then there is exactly one element $c \in G$ such that ac = b.


In [ ]:
## cool visualizations


In [ ]:
## Caley table


##### Abelian Group
A group $(G, *)$ is Abelian (or commutative) if the order of the operation does not matter:$$a * b = b * a \quad \text{for all } a, b \in G$$

These groups are the most "well-behaved". Basic addition $( \mathbb{Z}, +)$ is Abelian because $2+3 = 3+2$. However, most "rotational" groups are non-Abelian because the order in which you turn an object changes its final position.

##### Cyclic Group
A group is cyclic if every element in the group can be generated by repeatedly applying the group operation to a single element $g$ (called the generator).
* In multiplicative notation: $G = \{g^k \mid k \in \mathbb{Z}\}$.
* In additive notation: $G = \{kg \mid k \in \mathbb{Z}\}$.

Cyclic groups represent "loops." For example, the hours on a clock form a cyclic group of order 12; generated by 1 hour.

##### Symmetric Group
The Symmetric Group $S_n$ is the collection of all bijections (permutations) from a set of $n$ elements to itself.

- Every permutation can be uniquely written as a product of disjoint cycles. Disjoint cycles commute.

- A transposition is a cycle of length 2 (a simple swap). Any permutation can be expressed as a product of transpositions (e.g., $(1 \ 2 \ 3) = (1 \ 3)(1 \ 2)$). While the specific transpositions used may vary, the parity (even or odd) of the number of transpositions is always fixed for a given permutation.

- The order of a permutation is the smallest positive integer $k$ such that $\sigma^k = e$. For a permutation written in disjoint cycles, the order is the Least Common Multiple (LCM) of the lengths of those cycles.

- For $\sigma, \tau \in S_n$, the conjugate of $\sigma$ by $\tau$ is given by $\tau \sigma \tau^{-1}$. In $S_n$, conjugation simply "renames" the elements within the cycle structure of $\sigma$ according to the mapping of $\tau$.

- If $\sigma$ is a cycle of length $k$, then $\sigma^m$ is also a permutation where elements "jump" $m$ steps at a time. If $m$ and $k$ are coprime, $\sigma^m$ remains a single cycle of length $k$.

```
Conjugacy classes

Alternating groups ($A_n$)
```

##### Theorem: Parity of Transpositions
If an element $\sigma \in S_n$ is written as a product of transpositions in two ways:$$\sigma = \sigma_1 \sigma_2 \dots \sigma_k = \tau_1 \tau_2 \dots \tau_l$$

The Parity Theorem establishes that every rearrangement is either "Even" or "Odd." This creates a structural wall in the group, separating $S_n$ into the Alternating Group ($A_n$) and the rest. This distinction is the reason why, in a Rubik's Cube, you can never swap just two corners without affecting something else, you are constrained by the parity of the group. 

Lean Reference: See theorem transposition_parity in AA_Proofs.lean

##### Permutation Group
A group whose elements are permutations (rearrangements) of a set $M$, and the operation is function composition. The group of all permutations of $n$ elements is the Symmetric Group ($S_n$) and has $n!$ elements.

Cayley's Theorem proves that every finite group can be represented as a permutation group.


##### Dihedral Group
The group of symmetries of a regular $n$-sided polygon. It consists of:
* $n$ rotational symmetries.
* $n$ reflectional symmetries.
* Total size (order) is $2n$.

Dihedral groups are more complex types of groups because they are non-Abelian (for $n > 2$). Reflecting a square and then rotating it results in a different orientation than rotating it and then reflecting it.

In [ ]:
## Exaples (for each)

## cool visualizations


### Subgroups 
A subgroup $H$ is a subset of $G$ that satisfies the group axioms under the same operation as $G$.

##### Lagrange's Theorem:
Let $G$ be a finite group and $H$ be a subgroup of $G$ ($H \le G$). Then the order of $H$ must divide the order of $G$:$$|H| \mid |G|$$Furthermore, the number of distinct left cosets of $H$ in $G$ (called the index $[G:H]$) is given by:$$[G:H] = \frac{|G|}{|H|}$$ groups have a rigid, interlocking structure. If your group has 12 elements, you can never have a subgroup of size 5, because 5 doesn't divide 12.

##### Conjugates
A subgroup $K$ is a conjugate of $H$ if there exists some $g \in G$ such that $K = gHg^{-1}$. Conjugation can be thought of as "viewing the subgroup from the perspective of a different element."

##### Normal subgroup 
($N \triangleleft G$): A subgroup is normal if it is invariant under conjugation, meaning $gNg^{-1} = N$ for all $g \in G$. Equivalently, its left and right cosets are identical ($gN = Ng$). Normal subgroups are the only subgroups you can "divide" by to form a Quotient Group.

##### Simple subgroup
A group is called "simple" if it has no normal subgroups other than the identity group and itself. Simple groups are the "prime numbers" of group theory. They cannot be broken down into smaller group pieces.


### Cosets
If $H$ is a subgroup of $G$ and $g \in G$, the left coset is $gH = \{gh : h \in H\}$ and the right coset is $Hg = \{hg : h \in H\}$. Cosets partition the group $G$ into equal-sized, non-overlapping pieces.


In [ ]:
## Exaples (for each)


### Group actions
A group action is a formal way of using a group $G$ to map elements of a set $X$. Formally, an action of $G$ on $X$ is a function $G \times X \to X$ (denoted $g \cdot x$) such that:
- Identity: $e \cdot x = x$ for all $x \in X$.
- Compatibility: $(gh) \cdot x = g \cdot (h \cdot x)$ for all $g, h \in G$.
- The orbit of an element $x \in X$ is the set of all places $x$ can be moved by the group: $orb(Gx) = \{g \cdot x \mid g \in G\}$. Orbits partition the set $X$.
- The stabilizer of $x$ is the subgroup of $G$ consisting of elements that leave $x$ fixed: $stab(G_x) = \{g \in G \mid g \cdot x = x\}$.
- Orbit-Stabilizer Theorem: For a finite group $G$, the size of the orbit times the size of the stabilizer equals the order of the group: $|G| = |orb(Gx)| \cdot |stab(G_x)|$.

##### Direct sum of groups
The Direct Sum (or Direct Product for finite groups) $G \oplus H$ is a way to build a larger group from two smaller ones.Here the elements are ordered pairs $(g, h)$ where $g \in G$ and $h \in H$. With operations that are component-wise multiplication: $(g_1, h_1) * (g_2, h_2) = (g_1g_2, h_1h_2)$. The identity would be $(e_G, e_H)$. 

### Homomorphisms 
A homomorphism is a map between two algebraic structures of the same type (such as two groups) that preserves the operations defined on them. If we have a map $f: G \to H$, it is a homomorphism if $f(a * b) = f(a) \cdot f(b)$ for all $a, b \in G$. Essentially, a homomorphism is a "structure-preserving" translation. It ensures that the result of performing an operation in the first group and then mapping it to the second is the same as mapping the individual elements first and then performing the operation in the second group.

### Isomorphisms
An isomorphism is a special type of homomorphism that establishes a perfect, one-to-one correspondence between two algebraic structures. For a map $f: G \to H$ to be an isomorphism, it must be a homomorphism that is also a bijection (both injective and surjective). While a general homomorphism might "shrink" or "collapse" a group (like a shadow), an isomorphism preserves every bit of information. If an isomorphism exists between $G$ and $H$, the two groups are said to be isomorphic, denoted $G \cong H$. 

In [ ]:
## Exaples (for each)


##### Quaternions
The Quaternions are a set of eight elements $\{\pm 1, \pm i, \pm j, \pm k\}$ governed by the following  multiplication rules:
- $i^2 = j^2 = k^2 = ijk = -1$
- $1$ is the identity element, and $-1$ commutes with all elements.
- Non-commutative Multiplication: $ij = k$, but $ji = -k$ $jk = i$, but $kj = -i$ $ki = j$, but $ik = -j$ 

Properties: It is Non-Abelian, the order of multiplication matters $(ij \neq ji)$. All subgroups of $Q_8$ are normal, yet $Q_8$ is not an Abelian group. A group where all subgroups are normal is called a Hamiltonian group. Center: The center of the group (elements that commute with everything) is $Z(Q_8) = \{1, -1\}$.

The Quaternion Group (denoted as $Q_8$) is one of the most famous examples of a non-Abelian group. It was discovered by Sir William Rowan Hamilton in 1843 as an extension of complex numbers into three-dimensional space.


In [ ]:
## interesting visualization